In [23]:
from __future__ import print_function
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import cv2

#from ipywidgets import interact, interactive, fixed, interact_manual
#import ipywidgets as widgets
#from matplotlib.patches import Ellipse
#def extents(f):
#  delta = f[1] - f[0]
#  return [f[0] - delta/2, f[-1] + delta/2]


# Forward model: specimen -> Intensity

## Specimen properties

### Case 1: retardance varies from 0 to full-wave

In [25]:
# coordinate grid
x = np.linspace(-1.5,1.5,1024)
xx, yy = np.meshgrid(x,x)

# retardance: increases from 0 to full-wave (2pi) over radius of 1, and then stays constant
retardance = 2*np.pi*np.sqrt(xx**2+yy**2)
retardance[retardance >2*np.pi] = 2*np.pi


# slowaxis assumed to rotate azimuthally. 
slowaxis = np.arctan2(yy,xx)%np.pi


# constant transmission and polarization
transmission=np.ones(xx.shape)
polarization=np.ones(xx.shape)

### Case 2: retardance constant at half-wave

In [13]:
# coordinate grid

x = np.linspace(-1.5,1.5,1024)
xx, yy = np.meshgrid(x,x)

# retardance: increases from 0 to full-wave (2pi) over radius of 1, and then stays constant
retardance = np.pi*np.ones(xx.shape)


# slowaxis assumed to rotate azimuthally. 
slowaxis = np.arctan2(yy,xx)%np.pi


# constant transmission and polarization
transmission=np.ones(xx.shape)
polarization=np.ones(xx.shape)

### Case 3: retardance constant at quarter-wave 

In [32]:
# coordinate grid

x = np.linspace(-1.5,1.5,1024)
xx, yy = np.meshgrid(x,x)

# retardance: increases from 0 to full-wave (2pi) over radius of 1, and then stays constant
retardance = 0.5*np.pi*np.ones(xx.shape)


# slowaxis assumed to rotate azimuthally. 
slowaxis = np.arctan2(yy,xx)%np.pi


# constant transmission and polarization
transmission=np.ones(xx.shape)
polarization=np.ones(xx.shape)

### Plot retardance, slow-axis, transmission, and polarization

In [ ]:
xyextent=[-1.5,1.5,-1.5,1.5]

fig1 = plt.figure(figsize=(20,20)) # create a figure with the default size 

ax1 = fig1.add_subplot(2,2,1) 
im=plt.imshow(retardance,cmap='gray',extent=xyextent, origin='lower');
plt.title('retardance')
ax1.figure.colorbar(im);

ax2=fig1.add_subplot(2,2,2)
im=plt.imshow(slowaxis,cmap='twilight', extent=xyextent, origin='lower'); # Need to adapt this look up table to represent orientation.
plt.title('slow axis')
ax2.figure.colorbar(im);

ax3=fig1.add_subplot(2,2,3)
im=plt.imshow(transmission,cmap='gray',extent=xyextent, origin='lower');
plt.title('transmission')
ax3.figure.colorbar(im);

ax4=fig1.add_subplot(2,2,4)
im=plt.imshow(polarization,cmap='gray',extent=xyextent, origin='lower');
plt.title('polarization')
ax4.figure.colorbar(im);


In [33]:
np.save('./tests/testData/reconData/simulated/quarter_wave/retardance.png', retardance)
np.save('./tests/testData/reconData/simulated/quarter_wave/slowaxis.png', slowaxis)
np.save('./tests/testData/reconData/simulated/quarter_wave/transmission.png', transmission)
np.save('./tests/testData/reconData/simulated/quarter_wave/polarization.png', polarization)


## Stokes parameters with right circularly polarized illumination

In [34]:
# Assumptions: right circularly polarized input. 
s0=transmission
s1=transmission*polarization*np.sin(2*slowaxis)*np.sin(retardance)
s2=-transmission*polarization*np.cos(2*slowaxis)*np.sin(retardance)
s3=transmission*polarization*np.cos(retardance)

fig2 = plt.figure(figsize=(20,20)) # create a figure with the default size 

ax1 = fig2.add_subplot(2,2,1) 
im=plt.imshow(s0,cmap='gray',extent=xyextent, origin='lower');
plt.title('s0')
ax1.figure.colorbar(im);

ax2=fig2.add_subplot(2,2,2)
im=plt.imshow(s1,cmap='gray',extent=xyextent, origin='lower'); # Need to adapt this look up table to represent orientation.
plt.title('s1')
ax2.figure.colorbar(im);

ax3=fig2.add_subplot(2,2,3)
im=plt.imshow(s2,cmap='gray',extent=xyextent, origin='lower');
plt.title('s2')
ax3.figure.colorbar(im);

ax4=fig2.add_subplot(2,2,4)
im=plt.imshow(s3,cmap='gray',extent=xyextent, origin='lower');
plt.title('s3')
ax4.figure.colorbar(im);


In [35]:
np.save('./tests/testData/rawData/simulated/quarter_wave/s0.png', s0)
np.save('./tests/testData/rawData/simulated/quarter_wave/s1.png', s1)
np.save('./tests/testData/rawData/simulated/quarter_wave/s2.png', s2)
np.save('./tests/testData/rawData/simulated/quarter_wave/s3.png', s3)


## Intensity images

In [36]:
height,width=s0.shape
s=np.zeros((4,height,width))
s[0,:,:]=s0;  s[1,:,:]=s1; s[2,:,:]=s2; s[3,:,:]=s3;

"""
Swing of 0.25 or quarter-wave leads to linear analysis states. 
Since S1=I0-I90 and S2=I45-I135, the intensity patterns generated by 
"""
chi=2*np.pi*0.25
StoIMatLC=np.array([ [1, 0, 0, -1],
                     [1, np.sin(chi), 0, -np.cos(chi)],
                     [1, 0, np.sin(chi), -np.cos(chi)],
                     [1, -np.sin(chi), 0, -np.cos(chi)],
                     [1, 0, -np.sin(chi), -np.cos(chi)]])
Svec=s.reshape(4,height*width)
Ivec=np.dot(StoIMatLC,Svec)
I=Ivec.reshape(5,height,width)
Iext=I[0,:,:]; I0=I[1,:,:]; I45=I[2,:,:]; I90=I[3,:,:]; I135=I[4,:,:];


In [30]:
fig3 = plt.figure(figsize=(20,30)) # create a figure with the default size

ax1 = fig3.add_subplot(3,2,1) 
im=plt.imshow(Iext,cmap='gray',extent=xyextent,origin='lower');
plt.title('Iext')
ax1.figure.colorbar(im);

ax2=fig3.add_subplot(3,2,3)
im=plt.imshow(I0,cmap='gray',extent=xyextent,origin='lower'); # Need to adapt this look up table to represent orientation.
plt.title('I0')
ax2.figure.colorbar(im);

ax3=fig3.add_subplot(3,2,4)
im=plt.imshow(I45,cmap='gray',extent=xyextent,origin='lower');
plt.title('I45')
ax3.figure.colorbar(im);

ax4=fig3.add_subplot(3,2,5)
im=plt.imshow(I90,cmap='gray',extent=xyextent,origin='lower');
plt.title('I90')
ax4.figure.colorbar(im);

ax5=fig3.add_subplot(3,2,6)
im=plt.imshow(I135,cmap='gray',extent=xyextent,origin='lower');
plt.title('I135')
ax4.figure.colorbar(im);


In [37]:
np.save('./tests/testData/rawData/simulated/quarter_wave/Iext', Iext)
np.save('./tests/testData/rawData/simulated/quarter_wave/I0', I0)
np.save('./tests/testData/rawData/simulated/quarter_wave/I45', I45)
np.save('./tests/testData/rawData/simulated/quarter_wave/I90', I90)
np.save('./tests/testData/rawData/simulated/quarter_wave/I135', I135)